In [ ]:
#train.csv is uploaded from local

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/train.csv')

In [5]:
print(df.head())

                    qid                                      question_text  \
0  00002165364db923c7e6  How did Quebec nationalists see their province...   
1  000032939017120e6e44  Do you have an adopted dog, how would you enco...   
2  0000412ca6e4628ce2cf  Why does velocity affect time? Does velocity a...   
3  000042bf85aa498cd78e  How did Otto von Guericke used the Magdeburg h...   
4  0000455dfa3e01eae3af  Can I convert montra helicon D to a mountain b...   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  


In [6]:
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
def clean_texts(text) :
  text = text.lower();
  text = re.sub(r'[^a-z0-9\s]', '', text)
  return text


df['cleaned_text'] = df['question_text'].apply(clean_texts)

In [8]:
tokenizer = Tokenizer(num_words=20000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['cleaned_text'])
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])

In [9]:
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [1]:
!wget -O glove.GB.zip 'http://nlp.stanford.edu/data/glove.6B.zip'

--2025-06-07 11:42:14--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-06-07 11:42:14--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-06-07 11:42:14--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.GB.zip’

glov

In [2]:
!unzip -q glove.GB.zip

In [13]:
embedding_index={}
with open('glove.6B.100d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector


embedding_dim = 100
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [26]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense,Input
from tensorflow.keras.models import Model

In [21]:
input_layer = Input(shape=(max_length,))

In [22]:
embed_layer = Embedding(input_dim=len(word_index)+1,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=max_length,
                        trainable=False)(input_layer)

In [23]:
lstm_layer = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(embed_layer)
lstm_dense1 = Dense(64, activation='relu')(lstm_layer)
lstm_dense2 = Dense(32, activation='relu')(lstm_dense1)
output_layer = Dense(1, activation='sigmoid')(lstm_dense2)


In [29]:
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 132)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 132, 100)       │    24,724,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,852,517 (94.80 MB)

 Trainable params: 127,617 (498.50 KB)

 Non-trainable params: 24,724,900 (94.32 MB)

In [33]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
labels = df['target'].values

x_train, x_val, y_train, y_val = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [34]:
model.fit(x_train, y_train, epochs=5, batch_size=256, validation_data=(x_val, y_val))

Epoch 1/5
4082/4082 ━━━━━━━━━━━━━━━━━━━━ 1706s 416ms/step - accuracy: 0.9375 - loss: 0.2393 - val_accuracy: 0.9393 - val_loss: 0.2300
Epoch 2/5
4082/4082 ━━━━━━━━━━━━━━━━━━━━ 1708s 418ms/step - accuracy: 0.9381 - loss: 0.2327 - val_accuracy: 0.9393 - val_loss: 0.2290
Epoch 3/5
4082/4082 ━━━━━━━━━━━━━━━━━━━━ 1742s 414ms/step - accuracy: 0.9380 - loss: 0.2328 - val_accuracy: 0.9393 - val_loss: 0.2291
Epoch 4/5
4082/4082 ━━━━━━━━━━━━━━━━━━━━ 1670s 409ms/step - accuracy: 0.9380 - loss: 0.2329 - val_accuracy: 0.9393 - val_loss: 0.2289
Epoch 5/5
4082/4082 ━━━━━━━━━━━━━━━━━━━━ 1701s 409ms/step - accuracy: 0.9380 - loss: 0.2328 - val_accuracy: 0.9393 - val_loss: 0.2290


In [35]:
loss, accuracy = model.evaluate(x_val, y_val)
print(f'Validation Accuracy: {accuracy}')

8164/8164 ━━━━━━━━━━━━━━━━━━━━ 643s 79ms/step - accuracy: 0.9392 - loss: 0.2293
Validation Accuracy: 0.9393013715744019
